In [78]:
## GLOBAL VARIABLES
field_size = 10               # NxN, always square field
user_record = [[], []]       # list of lists with the turn record
win_line = []                # list of lists, win conditions that are still possible
field_pos_label = []         # position label for whole play field
available_pos_label = []     # position label for empty spot

In [79]:
# display function
def build_empty_field_v0(field_size):
      """Generates a grid of the given size"""
      row_up = "┌ - " + "- - " * (field_size - 2) + "- - ┐" + "\n" \
            + "|   " + "|   " * (field_size - 2) + "|   |" + "\n" \
            + "| - " + "+ - " * (field_size - 2) + "+ - |" + "\n" 
      row_mid = \
            "|   " + "|   " * (field_size - 2) + "|   |" + "\n" \
            + "| - " + "+ - " * (field_size - 2) + "+ - |" + "\n" 
      row_bot = \
            "|   " + "|   " * (field_size - 2) + "|   |" + "\n" \
            + "└ - " + "- - " * (field_size - 2) + "- - ┘" + "\n" 

      field = row_up + row_mid*(field_size-2) + row_bot
      return field

# Demo
field = build_empty_field_v0(field_size)        
print(field)

┌ - - - - - - - - - - - - - - - - - - - ┐
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
| - + - + - + - + - + - + - + - + - + - |
|   |   |   |   |   |   |   |   |   |   |
└ - - - - - - - - - - - - - - - - - - - ┘



In [80]:
def build_empty_field(field_size):
    row_up = "┌ - " + "- - " * (field_size - 2) + "- - ┐" + "\n" 
    row_inter = "| - " + "+ - " * (field_size - 2) + "+ - |" + "\n"
    row_bott = "└ - " + "- - " * (field_size - 2) + "- - ┘"
    numbers = [str(n).center(3, ' ') for n in range(1, field_size**2+1)]
    numbers = [numbers[i*field_size:(i+1)*field_size] for i in range(field_size)]
    rows_w_numbers = [['|'+number for number in row] for row in numbers]
    rows_w_numbers = [''.join(row)+'|\n' for row in rows_w_numbers] 

    field = row_up 
    for row_w_numbers in rows_w_numbers:
        field += row_w_numbers
        field += row_inter
    field = field.removesuffix(row_inter)
    field += row_bott
    return field

# Demo
field = build_empty_field(field_size)
print(field)

┌ - - - - - - - - - - - - - - - - - - - ┐
| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10|
| - + - + - + - + - + - + - + - + - + - |
| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20|
| - + - + - + - + - + - + - + - + - + - |
| 21| 22| 23| 24| 25| 26| 27| 28| 29| 30|
| - + - + - + - + - + - + - + - + - + - |
| 31| 32| 33| 34| 35| 36| 37| 38| 39| 40|
| - + - + - + - + - + - + - + - + - + - |
| 41| 42| 43| 44| 45| 46| 47| 48| 49| 50|
| - + - + - + - + - + - + - + - + - + - |
| 51| 52| 53| 54| 55| 56| 57| 58| 59| 60|
| - + - + - + - + - + - + - + - + - + - |
| 61| 62| 63| 64| 65| 66| 67| 68| 69| 70|
| - + - + - + - + - + - + - + - + - + - |
| 71| 72| 73| 74| 75| 76| 77| 78| 79| 80|
| - + - + - + - + - + - + - + - + - + - |
| 81| 82| 83| 84| 85| 86| 87| 88| 89| 90|
| - + - + - + - + - + - + - + - + - + - |
| 91| 92| 93| 94| 95| 96| 97| 98| 99|100|
└ - - - - - - - - - - - - - - - - - - - ┘


In [64]:
numbers = [str(n).center(3, ' ') for n in range(1, field_size**2+1)]
numbers = [numbers[i*field_size:(i+1)*field_size] for i in range(field_size)]
rows_w_numbers = [['|'+number for number in row] for row in numbers]
rows_w_numbers = [''.join(row)+'|' for row in rows_w_numbers] 

rows_w_numbers

['| 1 | 2 | 3 |', '| 4 | 5 | 6 |', '| 7 | 8 | 9 |']

In [56]:
_ = [list(item) for item in list(field.splitlines())]
numbers = [list(str(n).center(3, ' ')) for n in range(1, field_size**2+1)]
numbers = [numbers[i*field_size:(i+1)*field_size] for i in range(field_size)]
numbers

[[[' ', '1', ' '], [' ', '2', ' '], [' ', '3', ' ']],
 [[' ', '4', ' '], [' ', '5', ' '], [' ', '6', ' ']],
 [[' ', '7', ' '], [' ', '8', ' '], [' ', '9', ' ']]]

In [48]:
#''.join(list(field.splitlines()[1]))
_ = [list(item) for item in list(field.splitlines())]
numbers = [list(str(n).center(3, ' ')) for n in range(1, field_size**2+1)]
numbers = [numbers[i*field_size:(i+1)*field_size] for i in range(field_size)]
position_on_screen = # 1:4, 5:
for row_center, number_row in zip(_[1:field_size:2], numbers):
    for number in numbers:
        number = str(number).center(3, ' ')
    
        

# _[1][1:4] = '324'
# ''.join(_[1])

'|324|   |   |   |   |   |'

' 56'

In [ ]:
def render_field(field, ):
    """Populates the field with already placed symbols
    (Xs and Os) from the turn record"""
    pass

In [14]:
def init_field(field_size_n, user_history=None):
    """ Initialize the play field
    Args:
        field_size_n (int): a general play field nxn
        user_history (list): list of two user position history:
              [[usr1_pos1, usr2_pos2, usr1_pos3], [usr2_pos1, usr2_pos2]]].
              Defaults to None.
    """
    field_size = field_size_n
    field_pos_label = list(range(1, 1+field_size**2))
    if user_history == None:
        user_record = [[], []]
        available_pos_label = field_pos_label
    else:
        user_record = user_history
        ## check consistence
        if len(user_record[0]) != len(set(user_record[0])):
               print("warning: duplicate elements within user_history[0]")
               exit(0)
        if len(user_record[1]) != len(set(user_record[1])):
               print("warning: duplicate elements within user_history[1]")
               exit(0)
        if user_record[0] not in field_pos_label:
               print("warning: label overflow in user_history[0]")
               exit(0)
        if user_record[1] not in field_pos_label:
               print("warning: label overflow in user_history[1]")
               exit(0)
        available_pos_label = list(set(field_pos_label)
                                   - set(user_record[0])
                                   - set(user_record[1]))
    win_line = []
    tmp = list(range(field_size))
    for i in range(field_size):
        win_line.append([1+x+i*field_size for x in tmp])
        win_line.append([1+x*field_size+i for x in tmp])
    win_line.append([1+x*field_size+x for x in tmp])
    win_line.append([(1+x)*field_size-x for x in tmp])
    return user_record, win_line, available_pos_label

In [15]:
## global
if len(user_record[0]) == len(user_record[1]):
    whose_turn_is_it = 1
else:
    whose_turn_is_it = 0

def validate(available_pos_label):
    # avaiable_pos_label = set(field_pos_label) - set(user_record[0]) \
    #                     - set(user_record[1])
    if len(available_pos_label) == 0:
          print('warning: no available position to continue game!')
          print('game over!')
    turn_valid = False
    while not turn_valid:
        if whose_turn_is_it:
        # means it is the first player's turn
            turn = int(input("Player-1 (X): Please choose one available position label"))
            if turn in available_pos_label:
                 turn_valid = True
                 user_record[0].append(turn)
        else:
        # means it is the second player's turn
            turn = int(input("Player-2 (O): Please choose one available position label"))
            if turn in available_pos_label:
                 turn_valid = True
                 user_record[1].append(turn)

In [16]:
validate([1,5,9])

In [17]:
user_record

[[9], []]

In [4]:
init_field(4, None)

([[], []],
 [[1, 2, 3, 4],
  [1, 5, 9, 13],
  [5, 6, 7, 8],
  [2, 6, 10, 14],
  [9, 10, 11, 12],
  [3, 7, 11, 15],
  [13, 14, 15, 16],
  [4, 8, 12, 16],
  [1, 6, 11, 16],
  [4, 7, 10, 13]],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])